# NYC Taxi Trip Duration Prediction

In [ ]:
# 📦 Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor
from math import radians, cos, sin, asin, sqrt


In [ ]:
# 📥 Load Dataset
# (Download from https://www.kaggle.com/competitions/nyc-taxi-trip-duration/data)
df = pd.read_csv('train.csv', nrows=50000)  # limit rows for performance
print(df.shape)
df.head()


In [ ]:
# 🧼 Data Cleaning
df = df.dropna()
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])

# Remove outliers in trip duration
df = df[(df['trip_duration'] > 60) & (df['trip_duration'] < 7200)]


In [ ]:
# 🧪 Feature Engineering
df['pickup_hour'] = df['pickup_datetime'].dt.hour
df['pickup_day'] = df['pickup_datetime'].dt.dayofweek

# Haversine distance function
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    return R * 2 * asin(sqrt(a))

df['distance_km'] = df.apply(lambda row: haversine(
    row['pickup_latitude'], row['pickup_longitude'],
    row['dropoff_latitude'], row['dropoff_longitude']), axis=1)


In [ ]:
# 🎨 EDA
sns.histplot(df['trip_duration'] / 60, bins=50, kde=True)
plt.title("Trip Duration (minutes)")
plt.show()

sns.scatterplot(x='distance_km', y='trip_duration', data=df)
plt.title("Trip Duration vs Distance")
plt.show()


In [ ]:
# 🧠 Modeling - Linear Regression & XGBoost
features = ['passenger_count', 'pickup_hour', 'pickup_day', 'distance_km']
X = df[features]
y = df['trip_duration']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🔹 Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

# 🔹 XGBoost
xgb = XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)


In [ ]:
# 📊 Evaluation
def print_metrics(y_true, y_pred, model_name):
    print(f"{model_name} Results:")
    print("MAE:", mean_absolute_error(y_true, y_pred))
    print("RMSE:", np.sqrt(mean_squared_error(y_true, y_pred)))
    print("-" * 30)

print_metrics(y_test, y_pred_lr, "Linear Regression")
print_metrics(y_test, y_pred_xgb, "XGBoost")
